In [19]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By

import re
import time
import pandas as pd
import numpy as np

In [49]:
import pandas as pd
from sqlalchemy import URL, create_engine

url_object = URL.create(
                "postgresql+psycopg2",
                host='steam-db.c5m99euxia00.us-east-1.rds.amazonaws.com',
                username='myuser',
                port=5432,
                password='secret_passwd',
                database='steam_db')

engine = create_engine(url_object)
engine.connect()
connection = engine.raw_connection()

sql = """
SELECT * FROM steam
"""

games_df = pd.read_sql(sql, con=connection)



C:\Users\paulj\AppData\Local\Temp\ipykernel_8704\2816684352.py:20: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  games_df = pd.read_sql(sql, con=connection)


In [52]:
games_df

,title,Unique_ID,url,tags,win_comp,mac_comp,linux_comp,review,percent_review,total_review,date_released,discount,price
0,S.T.A.L.K.E.R. 2: Heart of Chornobyl,1643320,https://store.steampowered.com/app/1643320/STA...,"[1695,1663,3835,1662,122,1774,4166]",windows,None,None,Not Rated,Not Rated,Not Rated,December 2023,No Discount,59.99
1,The Elder Scrolls Online: Necrom,2172860,https://store.steampowered.com/app/2172860/The...,"[122,19,21,128,1684,1695,1754]",windows,mac,None,Not Rated,Not Rated,Not Rated,"Jun 5, 2023",No Discount,39.99
2,Street Fighter 6,1364780,https://store.steampowered.com/app/1364780/Str...,"[19,21,4736,1773,4231,1743,4747]",windows,None,None,Not Rated,Not Rated,Not Rated,"Jun 1, 2023",No Discount,59.99
3,Etrian Odyssey II HD,1868170,https://store.steampowered.com/app/1868170/Etr...,"[21,122,4434,1720,3871,4191,4325]",windows,None,None,Not Rated,Not Rated,Not Rated,"May 31, 2023",No Discount,39.99
4,Etrian Odyssey III HD,1810820,https://store.steampowered.com/app/1810820/Etr...,"[21,122,4434,1720,3871,4191,4325]",windows,None,None,Not Rated,Not Rated,Not Rated,"May 31, 2023",No Discount,39.99
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9033,Apex Predator Pack,418710,https://store.steampowered.com/app/418710/Apex...,"[19,21]",windows,mac,linux,Not Rated,Not Rated,Not Rated,None,No Discount,2.99
9034,"Warhammer 40,000: Gladius - Lord of Skulls",870540,https://store.steampowered.com/app/870540/Warh...,[9],windows,None,linux,Not Rated,Not Rated,Not Rated,None,No Discount,2.99
9035,"Warhammer 40,000: Gladius - Relics of War - So...",870530,https://store.steampowered.com/app/870530/Warh...,[9],None,None,None,Not Rated,Not Rated,Not Rated,None,No Discount,4.99
9036,Battlefield 4™ Vehicle Shortcut Bundle,1315443,https://store.steampowered.com/app/1315443/Bat...,[19],windows,None,None,Not Rated,Not Rated,Not Rated,None,No Discount,17.99


In [53]:
# Grab data from datatable / csv
# games_df = pd.read_csv('../data/steam.csv')
unique_id_list = games_df['Unique_ID'].to_numpy()





service = Service('/users/paulj/chromedriver')
driver = webdriver.Chrome(service=service)
driver.get("https://store.steampowered.com/search/?sort_by=Released_DESC&filter=topsellers&supportedlang=english")

element = driver.find_element(By.ID, 'search_resultsRows')
row_dict = {}
outside_array = []

# scroll to bottom
# last_height = driver.execute_script("return document.body.scrollHeight")
# while True:
#     # Scroll down to the bottom.
#     driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

#     # Wait to load the page.
#     time.sleep(1.5)

#     # Calculate new scroll height and compare with last scroll height.
#     new_height = driver.execute_script("return document.body.scrollHeight")

#     if new_height == last_height:

#         break

#     last_height = new_height

for row in element.find_elements(By.CSS_SELECTOR, 'a'):
    inside_array = []

    unique_id = np.int64(row.get_attribute('data-ds-appid'))
    print(unique_id)


    # conditional to see if new game
    if unique_id not in unique_id_list:
        print('got new')
        inside_array.append(row.get_attribute('data-ds-appid'))
        inside_array.append(row.get_attribute('href'))
        inside_array.append(row.find_element(By.CLASS_NAME, 'title').get_attribute('innerHTML'))
        inside_array.append(row.get_attribute('data-ds-tagids'))

        if len(row.find_elements(By.CLASS_NAME, 'win')) > 0:
            inside_array.append("windows")
        else:
            inside_array.append("")
        if len(row.find_elements(By.CLASS_NAME, 'mac')) > 0:
            inside_array.append("mac")
        else:
            inside_array.append("")
        if len(row.find_elements(By.CLASS_NAME, 'linux')) > 0:
            inside_array.append("linux")
        else:
            inside_array.append("")
        
        if len(row.find_elements(By.CLASS_NAME, 'positive')) > 0 or len(row.find_elements(By.CLASS_NAME, 'mixed')) > 0 or len(row.find_elements(By.CLASS_NAME, 'negative')) > 0:
            review = row.find_element(By.CLASS_NAME, 'search_review_summary').get_attribute('data-tooltip-html')
            numbers = re.findall(r'\d+', review)
            inside_array.append(review.split('<br>')[0])
            inside_array.append(numbers[0] + '%' + ' of positive reviews')
            inside_array.append(numbers[1] + " total reviews")
        else:
            inside_array.append("Not Rated")
            inside_array.append("Not Rated")
            inside_array.append("Not Rated")

        inside_array.append(row.find_element(By.CLASS_NAME, 'search_released').get_attribute('innerHTML'))
        if len(row.find_element(By.CLASS_NAME, 'search_discount').get_attribute('innerHTML').strip()) > 0: 
            inside_array.append('-' + row.find_element(By.CLASS_NAME, 'search_discount').get_attribute('innerHTML').split('%')[-2][-2:] + '%')
        else:
            inside_array.append("No Discount")
        inside_array.append(row.find_element(By.CLASS_NAME, 'search_price').get_attribute('innerHTML').split('$')[-1].strip())
        
    else:
        print('got old')
        inside_array = games_df.loc[games_df['Unique_ID'] == unique_id].values.flatten().tolist()
    
    outside_array.append(inside_array)


1574360
got new
1643320
got old
2172860
got old
1364780
got old
1868170
got old
1810820
got old
1868180
got old
1485590
got old
1294810
got old
1774580
got old
1798020
got old
482400
got old
2109370
got old
1593030
got old
1888930
got old
2254450
got old
1677350
got old
1999770
got old
2050650
got old
820520
got old
1942660
got old
1904392
got old
2277860
got old
2130460
got old
2106840
got old
1214470
got old
1448440
got old
1945340
got old
1971650
got old
1326470
got new
1969900
got new
1677280
got old
1838290
got new
1016950
got old
2307430
got new
1815570
got new
2152560
got new
2149610
got new
2004670
got new
1999920
got new
2217402
got new
2321680
got new
2299510
got new
2277770
got new
1122340
got new
791200
got new
2212520
got new
2081400
got new
2276590
got new
2101080
got new


In [76]:
pd.DataFrame(outside_array, columns = ['title', 'tags', 'win_comp','mac_comp','linux_comp','review','percent_review','total_review', 'date_released','discount', 'price' ]).to_csv('../data/steam.csv', index=False)

In [54]:
outside_array

[['1574360',
  'https://store.steampowered.com/app/1574360/Uncharted_Waters_Origin/?snr=1_7_7_7000_150_1',
  'Uncharted Waters Origin',
  '[122,599,9,1754,1741,128,21725]',
  'windows',
  '',
  '',
  'Not Rated',
  'Not Rated',
  'Not Rated',
  '2023',
  'No Discount',
  'Free To Play'],
 ['S.T.A.L.K.E.R. 2: Heart of Chornobyl',
  1643320,
  'https://store.steampowered.com/app/1643320/STALKER_2_Heart_of_Chornobyl/?snr=1_7_7_7000_150_1',
  '[1695,1663,3835,1662,122,1774,4166]',
  'windows',
  None,
  None,
  'Not Rated',
  'Not Rated',
  'Not Rated',
  'December 2023',
  'No Discount',
  '59.99'],
 ['The Elder Scrolls Online: Necrom',
  2172860,
  'https://store.steampowered.com/app/2172860/The_Elder_Scrolls_Online_Necrom/?snr=1_7_7_7000_150_1',
  '[122,19,21,128,1684,1695,1754]',
  'windows',
  'mac',
  None,
  'Not Rated',
  'Not Rated',
  'Not Rated',
  'Jun 5, 2023',
  'No Discount',
  '39.99'],
 ['Street Fighter 6',
  1364780,
  'https://store.steampowered.com/app/1364780/Street_Fi

In [5]:
# scroll to bottom
last_height = driver.execute_script("return document.body.scrollHeight")
while True:
    # Scroll down to the bottom.
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load the page.
    time.sleep(3)

    # Calculate new scroll height and compare with last scroll height.
    new_height = driver.execute_script("return document.body.scrollHeight")

    if new_height == last_height:

        break

    last_height = new_height

3271

480042